In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import time
from sklearn.preprocessing import MinMaxScaler

def custom_mre(y_true, y_pred):
    """numerator = tf.reduce_mean(tf.abs(y_pred - y_true), axis=None)
    denominator = tf.reduce_mean(tf.abs(y_true), axis=None)
    mre = 100.0 * numerator / denominator
    return mre"""
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    relative_error = np.abs((y_true - y_pred) / y_true)
    
    mre = np.mean(relative_error) * 100.0
    
    return mre

In [7]:
# Cargar el archivo CSV con punto y coma como delimitador
data = pd.read_csv('Cluster0ReadyToNN.csv', delimiter=';')

# Crear un diccionario para almacenar los objetos scaler por grupo
scalers = {}

# Iterar sobre los grupos únicos en Column15
for group in data['Column15'].unique():
    # Filtrar datos por grupo
    group_data = data[data['Column15'] == group]

    # Seleccionar las columnas para normalización (las 13 primeras)
    features = group_data.iloc[:, :13]

    # Normalizar los datos con MinMaxScaler
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(features)

    # Almacenar el scaler en el diccionario
    scalers[group] = scaler

    # Actualizar el DataFrame con los datos normalizados
    data.loc[data['Column15'] == group, 'Column1':'Column13'] = normalized_data

In [8]:
# Ordenar el DataFrame por 'Column 14' de forma ascendente
data = data.sort_values(by='Column14')

# Dividir los datos en entrenamiento (70%) y temporal (30%)
train_temp_data, test_data = train_test_split(data, test_size=0.3, stratify=data['Column15'], random_state=0)
#train_temp_data, test_data = train_test_split(data, test_size=0.3, shuffle=False, random_state=0)

# Dividir el temporal en entrenamiento (70%) y validación (30%)
train_data, validation_data = train_test_split(train_temp_data, test_size=0.3, stratify=train_temp_data['Column15'], random_state=0)
#train_data, validation_data = train_test_split(train_temp_data, test_size=0.3, shuffle=False, random_state=0)

# Separar características (X) y columna objetivo (y)
X_train_norm = train_data.iloc[:, :12]
y_train_norm = train_data['Column13']
X_val_norm = validation_data.iloc[:, :12]
y_val_norm = validation_data['Column13']
X_test_norm = test_data.iloc[:, :12]
y_test_norm = test_data['Column13']

In [15]:
# Definir el modelo de la red neuronal
model = keras.Sequential()

# Agregar capa de entrada
model.add(keras.layers.Input(shape=(12,)))  # Ahora son 12 características

# Agregar capas ocultas con las neuronas especificadas
neuronas_ocultas = [40,60,80,80,70,30,70,70,30,70,20,60]
for neurons in neuronas_ocultas:
    model.add(keras.layers.Dense(neurons, activation='tanh'))

# Agregar capa de salida con 1 neurona
model.add(keras.layers.Dense(1, activation='tanh'))

# Definir el optimizador
optimizer = keras.optimizers.Nadam(epsilon=1E-8, learning_rate=0.000044)

# Compilar el modelo con el optimizador personalizado
model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])

# Resumen del modelo
model.summary()

# Configurar Early Stopping
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Métrica a monitorear (en este caso, la pérdida en el conjunto de validación)
    patience=20,  # Número de épocas sin mejora antes de detener el entrenamiento
    restore_best_weights=True  # Restaurar los mejores pesos del modelo cuando se detiene
)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 40)                520       
                                                                 
 dense_27 (Dense)            (None, 60)                2460      
                                                                 
 dense_28 (Dense)            (None, 80)                4880      
                                                                 
 dense_29 (Dense)            (None, 80)                6480      
                                                                 
 dense_30 (Dense)            (None, 70)                5670      
                                                                 
 dense_31 (Dense)            (None, 30)                2130      
                                                                 
 dense_32 (Dense)            (None, 70)               

In [16]:
# Entrenar el modelo con Early Stopping
epochs = 300  
batch_size = 256
# Comienza a medir el tiempo de entrenamiento
start_time = time.time()
#Entrenamiento del modelo
history = model.fit(X_train_norm, y_train_norm, epochs=epochs, batch_size=batch_size, validation_data=(X_val_norm, y_val_norm), callbacks=[early_stopping])
# Finaliza la medición del tiempo de entrenamiento
end_time = time.time()

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test_norm)
print(y_pred)

Epoch 1/300
327/327 [==============================] - 5s 5ms/step - loss: 0.0321 - mse: 0.0321 - val_loss: 0.0287 - val_mse: 0.0287
Epoch 2/300
327/327 [==============================] - 2s 5ms/step - loss: 0.0278 - mse: 0.0278 - val_loss: 0.0274 - val_mse: 0.0274
Epoch 3/300
327/327 [==============================] - 1s 5ms/step - loss: 0.0270 - mse: 0.0270 - val_loss: 0.0271 - val_mse: 0.0271
Epoch 4/300
327/327 [==============================] - 2s 5ms/step - loss: 0.0267 - mse: 0.0267 - val_loss: 0.0266 - val_mse: 0.0266
Epoch 5/300
327/327 [==============================] - 2s 5ms/step - loss: 0.0264 - mse: 0.0264 - val_loss: 0.0264 - val_mse: 0.0264
Epoch 6/300
327/327 [==============================] - 2s 5ms/step - loss: 0.0262 - mse: 0.0262 - val_loss: 0.0264 - val_mse: 0.0264
Epoch 7/300
327/327 [==============================] - 2s 5ms/step - loss: 0.0260 - mse: 0.0260 - val_loss: 0.0261 - val_mse: 0.0261
Epoch 8/300
327/327 [==============================] - 2s 5ms/step - 

In [17]:
# Crear un DataFrame con las predicciones desnormalizadas y los valores reales
resultados = pd.DataFrame({'Valor Real': y_test_norm.values.flatten(), 'Predicciones': y_pred.flatten()})
print(resultados)

# Agregar la columna de predicciones al conjunto de prueba
test_data['Predicted_Column13'] = y_pred.flatten()

# Crear un DataFrame para almacenar los resultados desnormalizados
desnormalized_test_data = test_data.copy()

# Desnormalizar 'Column1' a 'Column13' y 'Predicted_Column13' según la normalización por grupos
for group, scalerY in scalers.items():
    # Filtrar el conjunto de prueba correspondiente al grupo
    group_test_data = test_data[test_data['Column15'] == group]

    # Seleccionar las columnas normalizadas para desnormalizar
    normalized_features = group_test_data[['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6', 'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column13', 'Predicted_Column13']]

    # Desnormalizar los datos utilizando el objeto scalerY correspondiente
    original_data = scalerY.inverse_transform(normalized_features)

    # Crear un DataFrame temporal para almacenar los datos desnormalizados
    temp_df = pd.DataFrame(original_data, columns=['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6', 'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column13', 'Predicted_Column13'])

    # Actualizar el DataFrame desnormalizado con los datos desnormalizados
    desnormalized_test_data.loc[desnormalized_test_data['Column15'] == group, ['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6', 'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column13', 'Predicted_Column13']] = temp_df.values

# Imprimir el conjunto de prueba después de la desnormalización
print(desnormalized_test_data)

# Eliminar todas las columnas excepto las últimas cuatro
resultados = desnormalized_test_data.iloc[:, -4:]

# Imprimir el conjunto de prueba después de la eliminación de columnas
print(resultados)

       Valor Real  Predicciones
0            41.0      0.226025
1            59.0      0.308004
2            23.0      0.143984
3           150.0      0.265041
4            34.0      0.101969
...           ...           ...
51171        65.5      0.168927
51172        10.0      0.071212
51173        13.0      0.088803
51174       145.0      0.300600
51175        85.0      0.507590

[51176 rows x 2 columns]


C:\Users\AdrianGilGamboa\AppData\Local\Temp\ipykernel_19808\1696851233.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[20.71956566 22.21303873 23.12504978 19.9679786  22.22203369 20.77547765
 20.765049   20.11592598 40.06992191 20.41897523 43.71335185 20.75099632
 42.32951242 17.82632804 18.62661885 19.97764014 23.80254489]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  desnormalized_test_data.loc[desnormalized_test_data['Column15'] == group, ['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6', 'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column13', 'Predicted_Column13']] = temp_df.values


        Column1  Column2  Column3  Column4  Column5  Column6  Column7  \
24905      58.0     66.0     62.0     56.0     77.0     60.0    170.0   
155320    101.0     39.0     24.0     28.0     65.0     32.0     28.0   
8745       30.0     30.0     29.0     31.0     31.0     31.0     29.0   
60462     193.0     59.0     48.0     87.0    186.0     59.0     47.0   
4241       35.0     29.0     37.0     31.0     29.0     32.0     32.0   
...         ...      ...      ...      ...      ...      ...      ...   
132964     37.0     35.0     33.0     35.0     34.0     36.0     21.0   
130033      5.0      8.0      7.0     11.0      1.0      7.0      8.0   
124375    137.0    120.0     63.0     60.0     54.0     73.0      6.0   
50855     185.0     32.0     49.0     67.0     51.0     37.0     56.0   
5442       29.0     65.0     44.0     31.0     50.0     69.0     48.0   

        Column8  Column9  Column10  Column11  Column12  Column13 Column14  \
24905      69.0     66.0      48.0      53.0  

In [18]:
# Obtener y_test_norm de la primera columna de resultados
y_test_norm = resultados['Column13']

# Obtener y_pred de la última columna del conjunto de prueba después de la desnormalización
y_pred = desnormalized_test_data['Predicted_Column13']

# Imprimir y_test_norm y y_pred
print("y_test_norm:", y_test_norm)
print("y_pred:", y_pred)


# Imprimir el DataFrame
print(resultados)
# Guardar el DataFrame resultados en un archivo CSV
resultados.to_csv('resultadosWaterKNIMENorm12layersTanh_MSE.csv', index=False)

# Calcular RMSE con datos desnormalizados
rmse = np.sqrt(mean_squared_error(y_test_norm, y_pred))
print(f'RMSE en el conjunto de prueba: {rmse:.2f}')

# Calcular MAE con datos desnormalizados
mae = mean_absolute_error(y_test_norm, y_pred)
print(f'MAE en el conjunto de prueba: {mae:.2f}')

#Calcular MAPE con datos desnormalizados
print("Mean absolute percentage error (MAPE): %f" % mean_absolute_percentage_error(y_test_norm, y_pred))

# Calcula la duración del entrenamiento en segundos
training_duration = end_time - start_time
# Imprime el tiempo de entrenamiento en segundos y en formato de horas, minutos y segundos
print(f'Tiempo de entrenamiento: {training_duration:.2f} segundos')
print(f'Tiempo de entrenamiento (HH:MM:SS): {int(training_duration // 3600)}:{int((training_duration % 3600) // 60)}:{int(training_duration % 60)}')

y_test_norm: 24905      41.0
155320     59.0
8745       23.0
60462     150.0
4241       34.0
          ...  
132964     65.5
130033     10.0
124375     13.0
50855     145.0
5442       85.0
Name: Column13, Length: 51176, dtype: float64
y_pred: 24905      55.451691
155320     52.588671
8745       28.351058
60462     137.462078
4241       26.765779
             ...    
132964     20.729962
130033     54.694163
124375     17.721967
50855     140.817262
5442       74.834867
Name: Predicted_Column13, Length: 51176, dtype: float64
        Column13 Column14  Column15  Predicted_Column13
24905       41.0   2005/2      8751           55.451691
155320      59.0   2006/3     56704           52.588671
8745        23.0   2015/2      3059           28.351058
60462      150.0   2011/3     20496          137.462078
4241        34.0   2009/4      1242           26.765779
...          ...      ...       ...                 ...
132964      65.5   2006/3     46569           20.729962
130033      10.0   200

In [19]:
# Convertir la columna 'Column14' para extraer el año
resultados['Año'] = resultados['Column14'].str.split('/').str[0].astype(int)

# Agrupar por año y calcular la suma de reales y predicciones
suma_anual = resultados.groupby('Año').agg({
    'Column13': 'sum',  # Suma de valores reales
    'Predicted_Column13': 'sum'  # Suma de predicciones
}).reset_index()

# Agregar una columna de MAPE por fila
suma_anual['MAPE_fila'] = (
    (abs(suma_anual['Column13'] - suma_anual['Predicted_Column13']) / suma_anual['Column13']) * 100
)

# Calcular la media de la columna MAPE
media_mape = suma_anual['MAPE_fila'].mean()

# Mostrar el DataFrame actualizado y la media
print(suma_anual)
print(f"Media de MAPE (por fila): {media_mape:.2f}%")

     Año       Column13  Predicted_Column13  MAPE_fila
0   2002  104265.678499       101239.084875   2.902771
1   2003  177491.021691       166279.109569   6.316890
2   2004  164849.403889       163826.212655   0.620682
3   2005  159059.300841       158585.406181   0.297936
4   2006  131381.875214       142057.003440   8.125267
5   2007  131223.654627       137741.803255   4.967206
6   2008  122996.287059       135242.151161   9.956288
7   2009  128977.774184       131368.953769   1.853947
8   2010  117483.995234       125947.126567   7.203646
9   2011  118736.260722       130746.369651  10.114946
10  2012  122697.561296       135470.196498  10.409853
11  2013  109818.900063       125413.487569  14.200277
12  2014  104937.636563       121247.936305  15.542850
13  2015  109243.945117       123272.331669  12.841340
Media de MAPE (por fila): 7.53%
